# Example: VQE program

This tutorial will be demonstation of creating VQE Program as well as migration guide on how you can replicate IBM Quantum VQE custom runtime program.

Let's first get information on what is VQE runtime program and what inputs and outputs it has.

**Description** of runtime program is: Variational Quantum Eigensolver (VQE) to find the minimal eigenvalue of a Hamiltonian.

**Inputs**:

| name | type | description |
| ---- | ---- | ----------- |
| ansatz | object | A parameterized quantum circuit preparing the ansatz wavefunction for the VQE. It is assumed that all qubits are initially in the 0 state. | 
| initial_parameters|[array,string]|Initial parameters of the ansatz. Can be an array or the string ``'random'`` to choose random initial parameters.|
|operator|object|The Hamiltonian whose smallest eigenvalue we're trying to find. Should be PauliSumOp|
|method|str|The classical optimizer used in to update the parameters in each iteration. |


**Return values**

| name | type | description |
| ---- | ---- | ----------- |
|cost_function_evals|integer|The number of cost function (energy) evaluations.|
|optimal_parameters|null|Not supported at the moment, therefore ``None``.|
|optimal_point|array|The optimal parameter values found during the optimization. This is a numpy array.|
|optimal_value|number|The smallest value found during the optimization. Equal to the ``eigenvalue`` attribute. This is a float.|
|optimizer_evals|integer|The number of steps of the optimizer.|
|optimizer_history|object|A dictionary containing information about the function evaluations (not necessarily the actual parameter value!): the current evaluation count, the parameters, the energy and the standard deviation.|
|optimizer_time|number|The total time taken by the optimizer. This is a float.|

We will also add optional `QiskitRuntimeService` as an argument to use that to access real devices.


With that information we can start drafting our program implementation in `vqe.py` file.

What our program should do:
1. parse input arguments
2. create run_vqe function that accepts estimator instance, creates VQE and runs calculation
3. decide which estimator to use and run vqe
    - if runtime service was passed then create a session and run `run_vqe` function
    - if runtime service was not passed then use stantard qiskit estimator
4. save results from vqe

Roughly our VQE program will look like this

```python
# vqe.py

import ...

def run_vqe(
    initial_parameters,
    ansatz,
    operator,
    estimator,
    method
):
    ...

arguments = get_arguments()

service = arguments.get("service")
ansatz = arguments.get("ansatz")
operator = arguments.get("operator")
initial_parameters = arguments.get("initial_parameters") 
optimizer = ...

...

if service is not None:
    # if we have service we need to open a session and create estimator
    backend = arguments.get("backend", "ibmq_qasm_simulator")
    with Session(service=service, backend=backend) as session:
        estimator = Estimator(session=session, options=options) # qiskit_ibm_runtime.Estimator
        vqe_result = run_vqe( estimator=estimator, ...)
else:
    # if we do not have a service let's use standart local estimator
    estimator = QiskitEstimator() # qiskit.primitives.Estimator

vqe_result, callback_dict = run_vqe(
    initial_parameters=initial_parameters,
    ansatz=ansatz,
    operator=operator,
    estimator=estimator,
    method=method
)

save_result({
    "optimal_point": vqe_result.x.tolist(),
    "optimal_value": vqe_result.fun,
    "optimizer_evals": vqe_result.nfev,
    "optimizer_history": callback_dict.get("cost_history", []),
    "optimizer_time": callback_dict.get("_total_time", 0)
})


```

At this point we have our program implemented. Now we need to actually run it. But before let's prepare input arguments from our VQE program.

In [9]:
import numpy as np


from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

USE_RUNTIME_SERVICE = False

service = None
if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService()
    backend = "ibmq_qasm_simulator"

operator = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
)
ansatz = EfficientSU2(operator.num_qubits)
    
input_arguments = {
    "ansatz": ansatz,
    "operator": operator,
    "method": "COBYLA",
    "service": service
}

input_arguments

{'ansatz': <qiskit.circuit.library.n_local.efficient_su2.EfficientSU2 at 0x7f7e59b02ee0>,
 'operator': SparsePauliOp(['YZ', 'ZI', 'ZZ', 'XX'],
               coeffs=[ 0.398 +0.j, -0.398 +0.j, -0.0113+0.j,  0.181 +0.j]),
 'method': 'COBYLA',
 'service': None}

With arguments prepared we can create our quantum serverless client, setup provider and run our program

In [10]:
from quantum_serverless import QuantumServerless, Provider
import os

In [11]:
provider = Provider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In [12]:
from quantum_serverless import Program

program = Program(
    title="VQE",
    entrypoint="vqe.py",
    working_dir="./source_files/vqe/"
)

job = serverless.run(program, arguments=input_arguments)
job

<Job | f9ec298d-788b-429e-9a42-196d56954d24>

In [13]:
job.status()

'QUEUED'

In [14]:
job.result()

'{"optimal_point": [-0.1792519774478331, 3.3690480121526396, 1.6216764077351256, 6.143139398511981, 0.49676653977835655, 2.374640057808242, 2.866200187127509, 2.50489457612712, 4.51800489248977, 4.78248888078524, 3.9586762267597324, 1.546113269910704, 7.685938511231928, 3.5731988425909336, 5.8298921420484, 5.098391668394297], "optimal_value": -0.7029303876247736, "optimizer_evals": 333, "optimizer_history": [-0.04397814147886023, -0.21348287749873493, -0.22682388084126995, -0.0683804070674735, -0.1669472475207855, 0.12409922698608211, -0.15487584379443692, -0.327011760994988, -0.3363001318119229, -0.02402793432805797, -0.3611931556726908, -0.3243501135790306, -0.6175274347223443, -0.20340618574430092, -0.6123428914378356, -0.500483771804176, -0.6838933249049571, -0.6619993124369523, -0.35369252630659076, -0.6230531482647764, -0.6931105208984558, -0.6255650373224049, -0.49365214652959305, -0.692786557513045, -0.6251937967963399, -0.6268364701131495, -0.5144299664153748, -0.6541106245138